In [2]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup, NavigableString
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import json
import os
from dotenv import load_dotenv
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import TimeoutException
from seleniumwire.utils import decode as sw_decode

from webdriver_manager.chrome import ChromeDriverManager
from fake_headers import Headers
import time
import numpy as np
import re

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [3]:
def chrome_driver(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    # browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver

def is_social_media_link(url):
    # Regular expression patterns for matching social media URLs
    patterns = {
        'Facebook': r'https?://(www\.)?facebook\.com/[\w.]+/?',
        'X (Twitter)': r'https?://(www\.)?twitter\.com/[\w.]+/?',
        'Instagram': r'https?://(www\.)?instagram\.com/[\w.]+/?',
        'Youtube': r'https?://(www\.)?(youtube\.com|youtu\.be)/[\w.]+/?',
        'TikTok':r'https?://(www\.)?tiktok\.com/@[\w.]+/?'
    }
    # Check each social media pattern to see if it matches the URL
    for platform, pattern in patterns.items():
        if re.match(pattern, url):
            return True, platform
    
    # If no pattern matches, the URL is not recognized as a social media link
    return False, None

In [34]:

# Fetch the webpage
url = "https://www.laliga.com/en-US"
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all divs with the class 'p-forge-list-item'
container = soup.find('div', attrs={"class":"styled__ClubesHeaderContainer-sc-1azasvg-0 hEJfTo"})
list_items = container.find_all('a')
# Initialize a list to hold the concatenated URLs
teams = []
concatenated_urls = []
url_to_follow_items = {}
for item in list_items:
    # Find the first 'a' tag within a 3-stacked div hierarchy
    # Extract the 'href' attribute
    href = item.get('href')
    
    # Concatenate with the base URL
    concatenated_urls.append(href)

# Print the concatenated URLs
for parent_url in concatenated_urls:
    # Fetch the webpage
    try:
        driver = chrome_driver()
        driver.get(parent_url)
        html_content = driver.page_source

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all 'li' items with the class 'p-follow__item'
        socials = soup.find_all('a')

        # Initialize a list to hold the hrefs for this parent URL
        href_links = []

        # Extract the 'href' attribute from each 'li' item found
        for item in socials:
            href = item.get('href')
            if href:
                is_valid, platform = is_social_media_link(href)
                if is_valid:
                    href_links.append(href)

        #     script_tag = soup.find('script', id='__NEXT_DATA__')

        #     if script_tag:
        #         # Get the text content of the script tag
        #         script_content = script_tag.string
                
        #         # Load the JSON content into a dictionary
        #         next_data = json.loads(script_content)
                
        #         # Navigate through the dictionary to get to the 'accounts' dictionary
        #         accounts = next_data['props']['pageProps']['initialState']['global']['config']['social']['accounts']
        #         items = []
        #         for _, i in accounts.items():
        #             items.append(i['url'])
        #         href_links = items
        # Associate the list of hrefs with the parent URL in the dictionary
        url_to_follow_items[parent_url] = href_links
        driver.close()
    except Exception:
        print("check", parent_url)
# Print the dictionary
for team, hrefs in url_to_follow_items.items():
    print(f"{team}: {hrefs}")


https://www.athletic-club.eus/en: ['https://www.instagram.com/athleticclub/?hl=es', 'https://twitter.com/AthleticClub', 'https://www.tiktok.com/@athleticclub?lang=es', 'https://www.facebook.com/ATHLETICCLUB/?locale=es_ES', 'https://www.instagram.com/athleticclubfem/?hl=es', 'https://twitter.com/AthleticClubFem']
https://en.atleticodemadrid.com: ['https://twitter.com/atletienglish', 'https://www.facebook.com/AtleticodeMadrid', 'https://www.youtube.com/clubatleticodemadrid?sub_confirmation=1', 'http://instagram.com/atleticodemadrid/', 'https://www.tiktok.com/@atleticodemadrid?utm_source=web&utm_medium=pie&utm_campaign=tiktok&utm_content=en']
https://www.osasuna.es/: ['https://www.youtube.com/caosasunatv', 'https://www.facebook.com/Osasuna/', 'https://twitter.com/Osasuna', 'https://www.youtube.com/CAOsasunaTV', 'https://www.tiktok.com/@caosasuna', 'https://www.instagram.com/caosasuna/', 'https://www.youtube.com/caosasunatv', 'https://www.facebook.com/Osasuna/', 'https://twitter.com/Osasun

In [35]:
sorted_teams = dict(sorted(url_to_follow_items.items()))
sorted_teams

{'http://www.rayovallecano.es/': ['https://www.facebook.com/rvmoficial',
  'https://www.instagram.com/rayovallecano/',
  'https://www.tiktok.com/@rayovallecano',
  'https://twitter.com/RayoVallecano',
  'https://www.youtube.com/user/rayovallecanotv',
  'https://www.facebook.com/rvmoficial',
  'https://www.instagram.com/rayovallecano/',
  'https://www.tiktok.com/@rayovallecano',
  'https://twitter.com/RayoVallecano',
  'https://www.youtube.com/user/rayovallecanotv'],
 'http://www.rccelta.es/index.php/en': [],
 'http://www.rcdmallorca.es/en': ['https://www.facebook.com/RCDMallorcaOficial',
  'https://www.instagram.com/rcdmallorcaoficial/',
  'https://www.tiktok.com/@realmallorcaoficial?lang=en',
  'https://twitter.com/rcd_mallorca',
  'https://www.youtube.com/user/OficialRCDMallorca',
  'https://www.youtube.com/watch?v=eDhFT3xMjYI',
  'https://www.facebook.com/RCDMallorcaOficial',
  'https://www.instagram.com/rcdmallorcaoficial/',
  'https://www.tiktok.com/@realmallorcaoficial?lang=en',


In [36]:
len(sorted_teams)

20

In [4]:
ref = pd.read_excel("laliga.xlsx")

In [38]:
for index, row in ref.iterrows():
    try:
        temp = sorted_teams[row['link']]
        print(temp)
        columns = ['Instagram', 'Facebook', 'TikTok','X (Twitter)','Youtube']
        for j in columns:
            for i in temp:
                is_valid, platform = is_social_media_link(i)
                if is_valid:
                    ref.at[index, platform] = i
    except:
        continue
    # row['Youtube'] = ([i for i in value if "youtube" in str(i)] + [""])[0]
    # row['Instagram'] = ([i for i in value if "instagram" in str(i)] + [""])[0]
    # row['Facebook'] = ([i for i in value if "facebook" in str(i)] + [""])[0]
    # row['TikTok'] = ([i for i in value if "tiktok" in str(i)] + [""])[0]
    # row['X (Twitter)'] = ([i for i in value if "twitter" in str(i)] + [""])[0]


['https://instagram.com/deportivoalaves/', 'https://twitter.com/alaves', 'https://www.tiktok.com/@deportivoalaves?lang=es', 'http://www.facebook.com/deportivoalaves', 'http://www.youtube.com/user/deportivoalaves1921', 'https://instagram.com/deportivoalaves/', 'https://twitter.com/alaves', 'https://www.tiktok.com/@deportivoalaves?lang=es', 'http://www.facebook.com/deportivoalaves', 'http://www.youtube.com/user/deportivoalaves1921', 'https://instagram.com/deportivoalaves/', 'https://twitter.com/alaves', 'https://www.tiktok.com/@deportivoalaves?lang=es', 'http://www.facebook.com/deportivoalaves', 'http://www.youtube.com/user/deportivoalaves1921']
['https://www.instagram.com/athleticclub/?hl=es', 'https://twitter.com/AthleticClub', 'https://www.tiktok.com/@athleticclub?lang=es', 'https://www.facebook.com/ATHLETICCLUB/?locale=es_ES', 'https://www.instagram.com/athleticclubfem/?hl=es', 'https://twitter.com/AthleticClubFem']
['https://twitter.com/atletienglish', 'https://www.facebook.com/Atle

In [39]:
ref.to_excel('laliga.xlsx', index=False)

: 

In [8]:
#Facebook
def text_to_int(text):
    # Initialize a multiplier
    multiplier = 1
    
    # Check if 'M' is in the text, indicating millions
    if 'na' in text:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M (na) follower', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K (na) follower', '')
    else:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M followers', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K followers', '')
def chrome_driver1(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    # browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver
for index, row in ref.iterrows():
    driver = chrome_driver1()
    driver.get(row['Facebook'])
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    script_tags = soup.find_all('script', attrs={'data-content-len': True})

    # Iterate over these script tags
    script_tags = soup.find_all('script')

    # Iterate over these script tags
    for script in script_tags:
        data_content_len = script.get('data-content-len')
        if data_content_len and int(data_content_len) > 20000:
            try:
                # Parse the script tag's content as JSON
                script_json = json.loads(script.text)
                for i in script_json['require'][0][3][0]['__bbox']['require'][7][3][1]['__bbox']['result']['data']['user']['profile_header_renderer']['user']['profile_social_context']['content']:
                    if i['uri'].endswith("/followers/"):
                        nested_content = i['text']['text']
                        ref.at[index, 'Facebook (followers)'] = text_to_int(nested_content)
                print(row['Facebook'],nested_content)
                break  # Found the target script tag, no need to continue
            except (KeyError, IndexError, json.JSONDecodeError):
                continue
    driver.quit()
# wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds
# element_present = wait.until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href="https://www.facebook.com/Padres/followers/"]'))
# )

# # Now execute the script
# script = """
# var element = document.querySelector('a[href="https://www.facebook.com/Padres/followers/"]');
# return element ? element.textContent : 'Element not found';
# """
# followers_text = driver.execute_script(script)
# followers_text = text_to_int(followers_text)
# print(followers_text)

http://www.facebook.com/deportivoalaves 1.3M (na) follower
https://www.facebook.com/ATHLETICCLUB 2.7M (na) follower
https://www.facebook.com/AtleticodeMadrid 17M (na) follower
https://www.facebook.com/fcbarcelona 113M (na) follower
https://www.facebook.com/cadizclubdefutbol 3.4M (na) follower
https://www.facebook.com/rccelta 2.5M (na) follower
https://www.facebook.com/GetafeCF 2.2M (na) follower
http://www.facebook.com/gironafc 1.9M (na) follower
https://www.facebook.com/GRANADACF.es 1.1M (na) follower
http://www.facebook.com/udlaspalmasoficial 1.6M (na) follower
https://www.facebook.com/RCDMallorcaOficial 1.9M (na) follower
https://www.facebook.com/Osasuna/ 1.6M (na) follower
https://www.facebook.com/rvmoficial 382K (na) follower
https://www.facebook.com/realbetisbalompie/ 4.7M (na) follower
https://www.facebook.com/RealMadrid/ 121M (na) follower
https://www.facebook.com/RealSociedadFutbol 1.7M (na) follower
https://www.facebook.com/sevillafc.eng/ 5.8M (na) follower
https://www.facebo

In [9]:
ref

,Unnamed: 0,link,YouTube (subscribers),Instagram (followers),Facebook (followers),TikTok (followers),X (Twitter) (followers),Unnamed: 6,Instagram,Facebook,TikTok,X (Twitter),Youtube
0,Alavés,https://www.deportivoalaves.com/,22900.0,161981,NaN,2800000,251398.0,NaN,https://instagram.com/deportivoalaves/,http://www.facebook.com/deportivoalaves,https://www.tiktok.com/@deportivoalaves,https://twitter.com/alaves,https://www.youtube.com/channel/UC5yHOZPDde9RW...
1,Athletic Bilbao,https://www.athletic-club.eus/en,137000.0,914876,NaN,5400000,1016376.0,NaN,https://www.instagram.com/athleticclub/?hl=es,https://www.facebook.com/ATHLETICCLUB,https://www.tiktok.com/@athleticclub,https://twitter.com/AthleticClub,https://www.youtube.com/channel/UCUiLE_NqFKarA...
2,Atlético Madrid,https://en.atleticodemadrid.com,392000.0,16757458,NaN,15300000,875561.0,NaN,http://instagram.com/atleticodemadrid/,https://www.facebook.com/AtleticodeMadrid,https://www.tiktok.com/@atleticodemadrid,https://twitter.com/atletienglish,https://www.youtube.com/channel/UCuzKFwdh7z2GH...
3,Barcelona,https://www.fcbarcelona.com/en,16600000.0,125847988,NaN,32800000,48696232.0,NaN,https://www.instagram.com/fcbarcelona/,https://www.facebook.com/fcbarcelona,https://www.tiktok.com/@fcbarcelona,https://twitter.com/FCBarcelona,https://www.youtube.com/channel/UC14UlmYlSNiQC...
4,Cadiz CF,https://www.cadizcf.com/,43900.0,426664,NaN,7100000,252631.0,NaN,https://www.instagram.com/cadizclubdefutbol/,https://www.facebook.com/cadizclubdefutbol,https://www.tiktok.com/@cadizclubdefutbol,https://twitter.com/cadiz_cf,https://www.youtube.com/channel/UCok4Ffi59YJWy...
5,Celta Vigo,https://www.rccelta.es/,61700.0,465871,NaN,6700000,527046.0,NaN,https://www.instagram.com/rccelta/,https://www.facebook.com/rccelta,https://www.tiktok.com/@rccelta,https://twitter.com/RCCelta,https://www.youtube.com/channel/UCCJLVZYqRb_85...
6,Getafe,https://www.getafecf.com/,21900.0,511589,NaN,4500000,220884.0,NaN,https://www.instagram.com/getafecf/channel/,https://www.facebook.com/GetafeCF,https://www.tiktok.com/@getafecf,https://twitter.com/GetafeCF,https://www.youtube.com/channel/UC34fUqq4rxJc0...
7,Girona,https://www.gironafc.cat/en,26000.0,722787,NaN,3600000,NaN,NaN,https://www.instagram.com/gironafc,http://www.facebook.com/gironafc,https://www.tiktok.com/@gironafc,http://twitter.com/gironafc,https://www.youtube.com/channel/UC6x5gKUZpXuKD...
8,Granada,https://www.granadacf.es/en,25000.0,374767,NaN,2100000,436126.0,NaN,https://instagram.com/granadacf,https://www.facebook.com/GRANADACF.es,https://www.tiktok.com/@granadacf,https://twitter.com/GranadaCF,https://www.youtube.com/channel/UC7QSPSHjp6E8K...
9,Las Palmas,https://www.udlaspalmas.es/en,27000.0,279102,NaN,2500000,3707.0,NaN,http://instagram.com/udlaspalmasoficial,http://www.facebook.com/udlaspalmasoficial,https://www.tiktok.com/@udlp_oficial,http://www.twitter.com/UDLP_english,https://www.youtube.com/channel/UCVmpXogwdZ48V...
